# Pose estimation tutorials

目前的2D Pose estimation主要分为两类：第一类是单人pose estimation，目前主流方法以stacked hourglass或者convulution pose machine结构为基础为主，后续多次的state-of-the-art主要基于这两种结构的基础上再进行改动，多数以堆积一些新的trick提高性能为主。

到底什么是pose estimation？可以这么理解：对人体的姿态信息进行估计，通过人体骨架的形式表现出来。如下面这张图：![Image of Yaktocat](https://github.com/CMU-Perceptual-Computing-Lab/openpose/raw/master/doc/media/keypoints_pose_18.png)

这张图是MScoco上的关键点解析。在coco上，标注了18个关键点的信息，如上图所示。pose estimation就是通过对人体关键点的回归，得到人体关键点的位置，得到骨架的特征，从而实现人体姿态估计的效果。具体的效果可以参见这张图片：![Image of Yaktocat](http://wx3.sinaimg.cn/mw690/005uXRWzly1fudt46yeb8j30rg0fnb29.jpg)

根据我的调研，最早的基于深度学习的方法出自deeppose，通过多阶的回归网络得到关键点坐标。整体结构如下所示：![Image of Yakotocat](http://wx4.sinaimg.cn/mw690/005uXRWzly1fuduar1fpgj310m08vdkv.jpg)
这个结构就是直接回归关键点坐标。结构很简单，多个stage不停回归，起到refine的作用。通过多阶的回归逐渐逼近label的坐标，整个网络就是简单的实现了一个回归任务。但是直接回归坐标点有一定问题，就是不灵活，因为只是单独回归坐标点，而且模型的扩展性受限。因此目前的主流做法并没有采用直接回归坐标点的方式，而是采用了回归坐标点heatmap的方法，这个heatmap就是坐标点的高斯响应。我们不再回归点的坐标，而是回归heatmap，响应图长这个样子：
![Image of Yaktocat](http://wx3.sinaimg.cn/small/005uXRWzly1fuduifug19j303z04daal.jpg)
回归的heatmap我们取这个gaussian响应的峰值，把这个响应内的峰值作为回归到的keypoint，这样就完成了回归相应图->得到关键点的过程。在回归热图的时候，引入了一种衡量指标PCK，以及PCK衍生的PCKh@α等指标。粗略的解释来说，PCK就是你最后得到的关键点坐标落在了该位置的ground truth的一个可容忍的(h,w)范围内，就认为回归的是准确的，PCKh@α则需要是α(h,w)内，其中的h，w以头部附近的范围为准，α是加上的一个约束。在util中有相关的eval代码，需要的时候可以直接使用。关于高斯响应生成的代码，如下所示，完整的一些方法代码也可以去eval中根据需要查找和修改。

In [1]:
import numpy as np
from skimage.filters import gaussian

def _gaussian(inp):
    '''
    :param inp: 关节点的坐标，size为(h,w,ch)，ch为关键点数量
    :return: 生成的响应
    '''
    h, w, ch = inp.shape   # 读取输入的关键点坐标信息
    out = np.zeros_like(inp)   
    for i in range(ch):
        j = np.argwhere(inp[:, :, i] == 1)
        if len(j) == 0:
            out[:, :, i] = np.zeros((h, w))
            continue
        j = j[0]
        t = np.zeros((h, w))
        t[j[0], j[1]] = 1
        t = gaussian(t, sigma=5, mode='constant')   # 生成响应
        out[:, :, i] = t * (1 / t.max())   # 归一化处理
    return out

上面简单介绍了pose estimation的大体定义。实际上pose estimation的任务还分为单人的姿态估计和多人姿态估计，上述内容简单介绍了一些单人姿态估计的方法和发展，单人姿态估计已经较为成熟，目前的state-of-the-art出自self adversarial learning for pose estimation.paper地址：(https://arxiv.org/pdf/1707.02439.pdf)。目前多人pose estimation是主要发展的任务，近期相关工作比较多，方法也分为两种：自顶向下和自底向上。目前的state-of-the-art方法是自顶向下的，也就是先检测人再在检测到的人上检测关键点，之后加nms处理冗余信息或是连接一个refine的network进行调整。自底向上的bottom-up方法速度快，性能上略差，但是可以实现实时的多人姿态估计，典型代表工作就是CMU-pose，开源了openpose库，实际效果如下：![Image of Yaktocat](https://github.com/CMU-Perceptual-Computing-Lab/openpose/raw/master/doc/media/dance_foot.gif)

关于一些具体的方法，我会在后续的paper reading部分给出。姿态估计的一些基本知识和代码分析，也会在后续继续更新。